In [1]:
import pandas as pd
import numpy as np
import editdistance as edd

In [2]:
measured_data = pd.read_excel("elife-16965-supp1-v4.xlsx")
imputed_data = pd.read_excel("elife-16965-supp2-v4.xlsx")
measured_data = measured_data[["Variants", "Fitness"]]
imputed_data.columns = ["Variants", "Fitness"]
data = measured_data
#data = pd.concat([measured_data, imputed_data]).reset_index(drop=True)
idx = data["Fitness"].argmax()
top_seq = data.loc[idx, "Variants"]
print(data.loc[idx])

/home/hury/miniconda3/envs/fast-hit/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
Variants        FWAA
Fitness     8.761966
Name: 57022, dtype: object


In [3]:
def calc_hd(ref_seq, variants):
    return [edd.eval(ref_seq, variant) for variant in variants]

In [4]:
hd = calc_hd(top_seq, data["Variants"])
data["HD"] = hd

In [5]:
data_hd = [data.loc[data["HD"] == i] for i in range(1, 5)]

In [6]:
score = data["Fitness"]
print(f"min: {score.min()}, max: {score.max()}, 20-th: {np.percentile(score, 20)}, 50-th: {np.percentile(score, 50)}, 80-th: {np.percentile(score, 80)}, 95-th: {np.percentile(score, 95)}, WT: {1.0}, 99-th: {np.percentile(score, 99)}")

min: 0.0, max: 8.76196565571, 20-th: 0.000541666669587, 50-th: 0.00338374487421, 80-th: 0.0122202760744, 95-th: 0.305672600518, WT: 1.0, 99-th: 2.152372151259997


In [7]:
data_fitness = []
for data in data_hd:
    data1 = data.loc[(data["Fitness"] >= np.percentile(score, 20)) & (data["Fitness"] < np.percentile(score, 50))]
    data1 = data1.reset_index(drop=True)
    data_fitness.append(data1)

    data2 = data.loc[(data["Fitness"] >= np.percentile(score, 50)) & (data["Fitness"] < np.percentile(score, 80))]
    data2 = data2.reset_index(drop=True)
    data_fitness.append(data2)
    data3 = data.loc[(data["Fitness"] >= np.percentile(score, 80)) & (data["Fitness"] < np.percentile(score, 95))]
    data3 = data3.reset_index(drop=True)
    data_fitness.append(data3)
    data4 = data.loc[(data["Fitness"] >= np.percentile(score, 95)) & (data["Fitness"] < 1.0)]
    data4 = data4.reset_index(drop=True)
    data_fitness.append(data4)
    data5 = data.loc[(data["Fitness"] >= 1.0) & (data["Fitness"] < np.percentile(score, 99))]
    data5 = data5.reset_index(drop=True)
    data_fitness.append(data5)

In [8]:
np.random.seed(1)
starts = pd.DataFrame([data.loc[np.random.randint(0, len(data))] for data in data_fitness if not data.empty])
starts = starts.reset_index(drop=True)
starts

,Variants,Fitness,HD
0,FWRA,0.010963,1
1,FWWA,0.045681,1
2,FWSA,0.913611,1
3,FEAA,1.346233,1
4,KWAY,0.002538,2
5,FTAY,0.006895,2
6,FCWA,0.027873,2
7,MWSA,0.529660,2
8,FHSA,1.516252,2
9,KLNA,0.002965,3
